<a href="https://colab.research.google.com/github/lynnelynne99/python/blob/main/Fish_speech_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


> **완료된 작업은 (O)표시함 🙃**

런타임 유형 변경> GPU로 설정 필요


# 1. 데이터셋 준비

In [18]:
# Colab에서 Google Drive를 마운트(O)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
#(1) Fish Speech 저장소 클론 및 기본 설정(O)

!git clone https://github.com/fishaudio/fish-speech.git
%cd fish-speech

Cloning into 'fish-speech'...
remote: Enumerating objects: 5482, done.
remote: Counting objects: 100% (1561/1561), done.
remote: Compressing objects: 100% (306/306), done.
remote: Total 5482 (delta 1324), reused 1257 (delta 1255), pack-reused 3921 (from 3)
Receiving objects: 100% (5482/5482), 18.60 MiB | 12.88 MiB/s, done.
Resolving deltas: 100% (3639/3639), done.
/content/fish-speech


In [19]:
#(2) Hugging Face에서 모델 다운로드(o)

!huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5

Fetching 7 files: 100% 7/7 [00:00<00:00, 4460.67it/s]
/content/checkpoints/fish-speech-1.5


**(3) 데이터셋 구성**

`.wav` / `.lab` 파일 준비

In [20]:
#(4)음성 데이터 정제를 위해 설치(o)
!pip install fish-audio-preprocess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 49.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached tiktoken-0.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [22]:
# 기존에 SPK1 폴더에 있는 파일을 data-raw로 (생성 후 복사)(o)
!mkdir -p /content/data-raw
!cp -r /content/drive/MyDrive/fish-speech/data/SPK1/* /content/data-raw/

In [23]:
# 복사가 잘 되었는지 확인(O)
!ls -lh /content/data-raw

total 5.6M
-rw------- 1 root root  197 Feb 11 02:57 1.lab
-rw------- 1 root root 5.6M Feb 11 02:57 1.wav


In [24]:
# (4)에서 설치했던 fish-audio-preprocess로 음성 데이터 정제 작업 실행(O)
!fap loudness-norm data-raw data --clean

2025-02-11 02:57:58.602 | INFO     | fish_audio_preprocess.cli.loudness_norm:loudness_norm:74 - Found 1 files, normalizing loudness
Preparing tasks: 100% 1/1 [00:00<00:00, 18.91it/s]
Processing:   0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
Processing: 100% 1/1 [00:00<00:00,  5.03it/s]
2025-02-11 02:57:58.868 | INFO     | fish_audio_preprocess.cli.loudness_norm:loudness_norm:102 - Done!
2025-02-11 02:57:58.868 | INFO     | fish_audio_preprocess.cli.loudness_norm:loudness_norm:103 - Total: 1, Skipped: 0
2025-02-11 02:57:58.869 | INFO     | fish_audio_preprocess.cli.loudness_norm:loudness_norm:104 - Output directory: data




---



# 2. Semantic Token 추출, 데이터셋 패킹
`.wav` 파일을 VQGAN을 활용해 `.npy` 파일로 변환



In [25]:
# (1) VQGAN 체크포인트 다운로드(O)
!huggingface-cli download fishaudio/fish-speech-1.5 \
    --local-dir checkpoints/fish-speech-1.5

Fetching 7 files: 100% 7/7 [00:00<00:00, 4382.11it/s]
/content/checkpoints/fish-speech-1.5


In [26]:
!pip install hydra-core #hydra 모듈 설치
!pip install hydra-core loguru click numpy tqdm torch torchvision torchaudio # 추가 라이브러리 설치

In [29]:
# portaudio 라이브러리 설치
!sudo apt-get install portaudio19-dev
!pip install pyaudio
!pip install -e /content/fish-speech #fish_speech 다시 설치

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 2s (123 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEn

In [66]:
# (2) Semantic Token 추출
!python /content/fish-speech/tools/vqgan/extract_vq.py /content/data \
    --num-workers 1 --batch-size 16 \
    --config-name "firefly_gan_vq" \
    --checkpoint-path "/content/checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

2025-02-11 04:25:49.969 | INFO     | __main__:main:189 | RANK: 0 / 1 - Starting worker
Found 1 files
2025-02-11 04:25:49.970 | INFO     | __main__:main:200 | RANK: 0 / 1 - Processing 1/1 files
/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:445: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:630: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/finite_scalar_quantization.py:147: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.11/dist-packages/vector_quantize_pyto

In [46]:
!ls /content/fish-speech/data/SPK1/ # 이 경로에 1.lab  1.npy 가 있는지 확인(O)

1.lab  1.npy


In [64]:
# (3) Protobuf으로 데이터셋 패킹: 추출된 semantic token을 protobuf 형식으로 변환하여 훈련 데이터로 사용할 수 있도록 패킹하는 과정
!python /content/fish-speech/tools/llama/build_dataset.py \
    --input "/content/fish-speech/data" \
    --output "/content/fish-speech/data/protos" \
    --text-extension .lab \
    --num-workers 16

0it [00:00, ?it/s]
Loading /content/fish-speech/data: 1it [00:00, 983.42it/s]

Grouping /content/fish-speech/data: 100% 1/1 [00:00<00:00, 4934.48it/s]
2025-02-11 04:20:54.086 | INFO     | __main__:task_generator_folder:46 - Found 1 groups in /content/fish-speech/data, ['/content/fish-speech/data/SPK1']...
1it [00:00, 91.06it/s]
2025-02-11 04:20:54.125 | INFO     | __main__:main:165 - Finished writing 1 shards to /content/fish-speech/data/protos


# 3. LoRA를 사용한 Fine-tuning 시작
> LoRA 파인튜닝을 위한 필수 파일

1.   `.lab` 파일:
   *   텍스트 데이터를 포함하는 파일 (예: 보험 상담 문장)
   *   `SPK1/1.lab`, `SPK1/2.lab`, ... 형식
2.   `.npy` 파일:
   *   .wav 파일을 변환한 벡터 데이터
   *   VQGAN 또는 VQ-VAE 같은 오디오 인코더를 사용해 변환
   *   `SPK1/1.npy`, `SPK1/2.npy`, ... 형식


---

    1️⃣ .wav 파일은 모델 입력으로 직접 사용되지 않고, .npy로 변환된 뒤 사용됨
    - .wav → VQGAN 변환 → .npy (오디오 특징 벡터)
    - 모델은 직접 .wav를 학습하지 않고 .npy를 사용함

    2️⃣ 파인튜닝 시 모델이 학습하는 것은 .lab(텍스트)과 .npy(오디오 특징 벡터)
    - .lab: 텍스트 데이터 (음성이 어떤 문장을 의미하는지)
    - .npy: 변환된 음성 데이터 (오디오 임베딩)

    3️⃣ 따라서 .wav가 없어도 .npy가 제대로 생성되어 있으면 문제없이 진행 가능
    - .lab + .npy만 있으면 모델이 필요한 모든 정보를 학습 가능
    - .wav 없이도 LoRA 파인튜닝 수행 가능

In [51]:
!ls -lh /content/fish-speech/data/protos/


total 60K
-rw-r--r-- 1 root root 58K Feb 11 04:01 00000000.protos


In [107]:
!python /content/fish-speech/fish_speech/train.py --config-name text2semantic_finetune \
    project=my_fish_speech \
    +lora@model.model.lora_config=r_8_alpha_16


# --config-name text2semantic_finetune: text2semantic 모델을 fine-tuning
# project=my_fish_speech: 결과를 저장할 프로젝트명
# +lora@model.model.lora_config=r_8_alpha_16: LoRA를 적용한 Fine-tuning 수행

[2025-02-11 08:07:04,095][__main__][INFO] - [rank: 0] Instantiating datamodule <fish_speech.datasets.semantic.SemanticDataModule>
[2025-02-11 08:07:04,116][datasets][INFO] - PyTorch version 2.5.1+cu124 available.
[2025-02-11 08:07:04,116][datasets][INFO] - TensorFlow version 2.18.0 available.
[2025-02-11 08:07:04,117][datasets][INFO] - JAX version 0.4.33 available.
[2025-02-11 08:07:04,805][__main__][INFO] - [rank: 0] Instantiating model <fish_speech.models.text2semantic.lit_module.TextToSemantic>
[2025-02-11 08:07:04,818][fish_speech.models.text2semantic.llama][INFO] - [rank: 0] Override max_seq_len to 4096
[2025-02-11 08:07:04,988][fish_speech.models.text2semantic.llama][INFO] - [rank: 0] Loading model from checkpoints/fish-speech-1.5, config: DualARModelArgs(model_type='dual_ar', vocab_size=102048, n_layer=24, n_head=16, dim=1024, intermediate_size=4096, n_local_heads=2, head_dim=64, rope_base=1000000.0, norm_eps=1e-06, max_seq_len=4096, dropout=0.0, tie_word_embeddings=False, atten

# ZeroDivisionError: integer division or modulo by zero 에러


이는 데이터셋이 너무 적거나, 특정 연산에서 0이 나와서 발생하는 문제라고 함
> 아래는 .lab 파일이 부족해서 LoRA Fine-tuning이 실패하는 것 같아 실행한 내용

(.lab 파일을 더 생성하고, npy 파일을 그에 맞게 만들어줌. npy파일은 1.wav 파일(3분 분량)을 1/10로 나눠서 가져감)

In [75]:
!echo "고객님, 어떤 보험 상담을 도와드릴까요?" > /content/fish-speech/data/SPK1/2.lab
!echo "보험 청구를 진행하려면 몇 가지 서류가 필요합니다." > /content/fish-speech/data/SPK1/3.lab
!echo "긴급 출동 서비스가 필요하신가요?" > /content/fish-speech/data/SPK1/4.lab
!echo "현재 가입하신 보험 보장 내역을 확인해드릴까요?" > /content/fish-speech/data/SPK1/5.lab
!echo "보험료 할인 혜택이 적용될 수 있는지 확인해보겠습니다." > /content/fish-speech/data/SPK1/6.lab
!echo "사고 접수는 앱을 통해서도 가능합니다. 안내해드릴까요?" > /content/fish-speech/data/SPK1/7.lab
!echo "보험 갱신 시 변경된 사항이 있을 수 있으니 확인 부탁드립니다." > /content/fish-speech/data/SPK1/8.lab
!echo "고객님께 적합한 보험 상품을 추천해드릴 수 있습니다." > /content/fish-speech/data/SPK1/9.lab
!echo "추가 특약을 원하시면 옵션을 설명해드리겠습니다." > /content/fish-speech/data/SPK1/10.lab
!echo "최근 사고 이력이 있으신가요? 보험료에 영향을 줄 수 있습니다." > /content/fish-speech/data/SPK1/11.lab

In [76]:
!mkdir -p /content/fish-speech/data/SPK1/

import librosa
import numpy as np
import os

wav_path = "/content/data-raw/1.wav"
output_dir = "/content/fish-speech/data/SPK1/"

# 오디오 로드 (16kHz 샘플링)
y, sr = librosa.load(wav_path, sr=16000)

# 18초씩 나누기 (18초 * 16000Hz = 288000 samples)
segment_length = sr * 18

# 오디오를 여러 개 segment로 나누기
segments = [y[i : i + segment_length] for i in range(0, len(y), segment_length)]

# 기존에 존재하는 1.npy 이후 번호부터 시작
start_idx = 2 if os.path.exists(os.path.join(output_dir, "1.npy")) else 1

# lab 파일 리스트 (사전에 만들어진 텍스트 파일들)
lab_files = sorted([f for f in os.listdir(output_dir) if f.endswith(".lab")])

# segment 개수와 .lab 파일 개수를 맞추기 위해 조정
num_files = min(len(segments), len(lab_files))

for i in range(num_files):
    npy_filename = os.path.join(output_dir, f"{start_idx + i}.npy")
    np.save(npy_filename, segments[i])
    print(f"✅ Saved: {npy_filename}")

print("🎉 모든 .npy 파일이 정상적으로 저장되었습니다!")


✅ Saved: /content/fish-speech/data/SPK1/2.npy
✅ Saved: /content/fish-speech/data/SPK1/3.npy
✅ Saved: /content/fish-speech/data/SPK1/4.npy
✅ Saved: /content/fish-speech/data/SPK1/5.npy
✅ Saved: /content/fish-speech/data/SPK1/6.npy
✅ Saved: /content/fish-speech/data/SPK1/7.npy
✅ Saved: /content/fish-speech/data/SPK1/8.npy
✅ Saved: /content/fish-speech/data/SPK1/9.npy
✅ Saved: /content/fish-speech/data/SPK1/10.npy
✅ Saved: /content/fish-speech/data/SPK1/11.npy
✅ Saved: /content/fish-speech/data/SPK1/12.npy
🎉 모든 .npy 파일이 정상적으로 저장되었습니다!


# 확인용 셀

In [93]:
import numpy as np
import os

npy_dir = "/content/fish-speech/data/SPK1"
npy_files = [f for f in os.listdir(npy_dir) if f.endswith(".npy")]

for npy_file in npy_files:
    path = os.path.join(npy_dir, npy_file)
    data = np.load(path)

    if data.size == 0:
        print(f"⚠️ 경고: {npy_file} 파일이 비어 있습니다.")
    else:
        print(f"✅ {npy_file} 파일 정상 로드됨. 데이터 크기: {data.shape}")


✅ 3.npy 파일 정상 로드됨. 데이터 크기: (288000,)
✅ 11.npy 파일 정상 로드됨. 데이터 크기: (288000,)
✅ 2.npy 파일 정상 로드됨. 데이터 크기: (288000,)
✅ 4.npy 파일 정상 로드됨. 데이터 크기: (288000,)
✅ 8.npy 파일 정상 로드됨. 데이터 크기: (288000,)
✅ 5.npy 파일 정상 로드됨. 데이터 크기: (288000,)
✅ 10.npy 파일 정상 로드됨. 데이터 크기: (288000,)
✅ 6.npy 파일 정상 로드됨. 데이터 크기: (288000,)
✅ 9.npy 파일 정상 로드됨. 데이터 크기: (288000,)
✅ 7.npy 파일 정상 로드됨. 데이터 크기: (288000,)


In [95]:
!ls -l /content/fish-speech/data/SPK1/*.npy


-rw-r--r-- 1 root root 1152128 Feb 11 06:01 /content/fish-speech/data/SPK1/10.npy
-rw-r--r-- 1 root root 1152128 Feb 11 06:01 /content/fish-speech/data/SPK1/11.npy
-rw-r--r-- 1 root root 1152128 Feb 11 06:01 /content/fish-speech/data/SPK1/2.npy
-rw-r--r-- 1 root root 1152128 Feb 11 06:01 /content/fish-speech/data/SPK1/3.npy
-rw-r--r-- 1 root root 1152128 Feb 11 06:01 /content/fish-speech/data/SPK1/4.npy
-rw-r--r-- 1 root root 1152128 Feb 11 06:01 /content/fish-speech/data/SPK1/5.npy
-rw-r--r-- 1 root root 1152128 Feb 11 06:01 /content/fish-speech/data/SPK1/6.npy
-rw-r--r-- 1 root root 1152128 Feb 11 06:01 /content/fish-speech/data/SPK1/7.npy
-rw-r--r-- 1 root root 1152128 Feb 11 06:01 /content/fish-speech/data/SPK1/8.npy
-rw-r--r-- 1 root root 1152128 Feb 11 06:01 /content/fish-speech/data/SPK1/9.npy
